In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
width = 9
height = 9
n_mines = 10

In [3]:
stage = np.zeros((width, height, 3), dtype='int8')

In [4]:
def get_init_stage(width=9, height=9, n_mines=10):
    # set stage
    stage = np.zeros((width, height, 3), dtype='int8')
    # assign mines
    selected_long = np.zeros((width * height), dtype='bool')
    selected_long[np.random.choice(range(0, width * height), n_mines, replace=False)] = True
    selected = selected_long.reshape((width, height))
    stage[selected, 1] = 1
    # set neighbour mine counts
    mines_padded = np.zeros((width + 2, height + 2), dtype='int8')
    mines_padded[1:-1, 1:-1] = stage[:, :, 1]
    for x in range(width):
        for y in range(height):
            if stage[x, y, 1] == 0:
                stage[x, y, 2] = np.sum(mines_padded[x: x + 3, y: y + 3])
    return stage          

In [5]:
def print_stage(stage):
    return stage[:, :, 2] - stage[:, :, 1]

In [6]:
print_stage(get_init_stage())

array([[ 1,  1,  0,  0,  0,  1,  2,  3, -1],
       [-1,  1,  0,  0,  0,  1, -1, -1,  2],
       [ 1,  1,  0,  0,  0,  1,  2,  2,  1],
       [ 0,  0,  0,  0,  0,  1,  1,  2,  1],
       [ 0,  0,  0,  0,  1,  2, -1,  2, -1],
       [ 0,  0,  0,  0,  1, -1,  2,  2,  1],
       [ 0,  0,  0,  0,  1,  2,  2,  1,  0],
       [ 0,  1,  1,  1,  0,  1, -1,  2,  1],
       [ 0,  1, -1,  1,  0,  1,  1,  2, -1]], dtype=int8)

In [7]:
def show_visible(stage):
    return stage[:, :, 0] * print_stage(stage)

In [8]:
show_visible(get_init_stage())

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [9]:
def expand(stage, coord):
    mines_padded = np.zeros((width + 2, height + 2), dtype='int8') - 1
    mines_padded[1:-1, 1:-1] = stage[:, :, 1]
    n = []
    n.append((coord[0], coord[1] + 1))
    n.append((coord[0] + 1, coord[1]))
    n.append((coord[0] + 2, coord[1] + 1))
    n.append((coord[0] + 1, coord[1] + 2))
    return [(x[0] - 1, x[1] - 1) for x in n if mines_padded[x[0], x[1]] == 0
           and stage[x[0] - 1, x[1] - 1, 0] == 0]

In [10]:
def step_on(stage, coord):
    new_stage = stage.copy()
    new_stage[coord[0], coord[1], 0] = 1
    # if not mine, expand all (directly) neighbouring non-mine tiles
    if is_dead(new_stage):
        return False, new_stage
    elif stage[coord[0], coord[1], 2] == 0:
        eligible_neighbours = expand(new_stage, coord)
        for c in eligible_neighbours:
            new_stage[c[0], c[1], 0] = 1
            if new_stage[c[0], c[1], 2] == 0:
                eligible_neighbours += expand(new_stage, c)
        return True, new_stage
    else:
        return True, new_stage

In [11]:
def is_dead(stage):
    return np.sum(show_visible(stage) < 0)

In [177]:
def pretty(stage):
    pres = pd.DataFrame(print_stage(stage))
    pres = pres.replace(0, '.')
    for c in np.argwhere(stage[:, :, 0] == 0):
        pres.loc[c[0], c[1]] = '*'
    return pres

In [193]:
temp_stage = get_init_stage()

In [194]:
temp_printed = print_stage(temp_stage)
temp_printed

array([[ 0,  1, -1,  1,  0,  0,  1, -1,  1],
       [ 0,  1,  1,  1,  0,  0,  1,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  1,  1],
       [ 0,  0,  1,  2,  2,  1,  2, -1,  2],
       [ 0,  1,  2, -1, -1,  2,  3, -1,  3],
       [ 0,  1, -1,  4, -1,  2,  3, -1,  3],
       [ 0,  1,  1,  2,  1,  1,  2, -1,  2]], dtype=int8)

In [195]:
temp_printed[2, 2]

0

In [196]:
print(pretty(temp_stage))

   0  1  2  3  4  5  6  7  8
0  *  *  *  *  *  *  *  *  *
1  *  *  *  *  *  *  *  *  *
2  *  *  *  *  *  *  *  *  *
3  *  *  *  *  *  *  *  *  *
4  *  *  *  *  *  *  *  *  *
5  *  *  *  *  *  *  *  *  *
6  *  *  *  *  *  *  *  *  *
7  *  *  *  *  *  *  *  *  *
8  *  *  *  *  *  *  *  *  *


In [197]:
alive, temp_after = step_on(temp_stage, (2, 2))
print(alive)
print(pretty(temp_after))
# print(temp_after[:, :, 0])
# show_visible(temp_after)

True
   0  1  2  3  4  5  6  7  8
0  .  1  *  1  .  .  1  *  *
1  .  1  1  1  .  .  1  1  1
2  .  .  .  .  .  .  .  .  .
3  .  .  .  .  .  .  .  .  .
4  .  .  .  .  .  .  1  1  1
5  .  .  1  2  2  1  *  *  *
6  .  1  *  *  *  *  *  *  *
7  .  1  *  *  *  *  *  *  *
8  .  1  *  *  *  *  *  *  *


In [198]:
def is_won(stage):
    return np.sum(stage[:, :, 0]) == width * height - n_mines